In [24]:
import pandas as pd
from pathlib import Path
from typing import List

In [34]:
TRENDS = (
    "zero_trend",
    "constant_trend",
    "linear_trend",
    "nonlinear_trend"
)

MEASUREMENT_FILENAMES = [
    "gaps.csv",
    "minimum_observations.csv",
    "observations.csv",
    "simple_regrets.csv"
]


def get_testfn_dirs():
    current = Path('.')
    dirs = [p for p in current.iterdir() if p.is_dir() and p.name[0] != "."]
    return dirs


def get_trend_to_surrogate(function_trend_index, surrogate_trend_index):
    return f"{TRENDS[function_trend_index]}/{TRENDS[surrogate_trend_index]}_surrogate"

In [64]:
testfn_dirs = get_testfn_dirs()
strategy = "EI"
fti = 0
sti = 0
metric = MEASUREMENT_FILENAMES[0]
dataframes = dict()

for tf_dir in testfn_dirs:
    tf_name = tf_dir.name
    tf_gap_path = tf_dir / get_trend_to_surrogate(fti, sti) / strategy / metric
    df = pd.read_csv(tf_gap_path)
    # Drop dummy row
    df = df.iloc[1:]
    # Set index to be trial column
    df = df.set_index("trial")
    dataframes[tf_name] = df

In [70]:
trend_surrogate_pairs = dict()
for function_trend_index in range(len(TRENDS)):
    for surrogate_trend_index in range(len(TRENDS)):
        dfs_for_metric = dict()
        for tf_dir in testfn_dirs:
            path_to_metric = get_trend_to_surrogate(function_trend_index, surrogate_trend_index)
            metric_csv = tf_dir / path_to_metric / strategy / metric
            df = pd.read_csv(metric_csv)
            # Drop dummy row
            df = df.iloc[1:]
            # Set index to be trial column
            df = df.set_index("trial")
            # Grabs all metrics associated with current (function_trend_index, surrogate_trend_index) pair
            dfs_for_metric[tf_dir.name] = df
        trend_surrogate_pairs[(function_trend_index, surrogate_trend_index)] = dfs_for_metric
            

In [114]:




# suppose your list of DataFrames is:
pair = (3, 3)
dfs = list(trend_surrogate_pairs[pair].values())

# 1. Extract the GAP series from each and concatenate side–by–side
gap_matrix = pd.concat(
    [df for df in dfs],
    axis=0,
    keys=range(len(dfs))  # or use names/labels if you have them
)

# 2. Compute summary stats across the columns (i.e. across functions) for each trial
summary = pd.DataFrame({
    "mean_gap":   gap_matrix.mean(axis=0),
    "std_gap":    gap_matrix.std(axis=0),
    "median_gap": gap_matrix.median(axis=0),
})

# # 3. (Optional) if you want a “wide” table with functions as rows and stats as columns:
# # summary = gap_matrix.agg(["mean", "std", "median"], axis=1).rename_axis("trial")

# print(summary.head())

In [116]:
trend_surrogate_pairs[pair]

{'mccormick':          1    2    3    4    5    6    7    8    9   10  ...   92   93   94  \
 trial                                                    ...                  
 1.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 2.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 3.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 4.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 5.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 6.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 7.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 8.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 9.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 10.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 11.0   0.0  0.0  0.0  0.0 